In [5]:
import os.path as op
import numpy as np
import re
from sentimentanalysis import *
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict

In [6]:
# Load data
print("Loading dataset")

from glob import glob
filenames_neg = sorted(glob(op.join('/Users/mehdiregina/Documents/jupyter/MDI343/TPTextMining', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('/Users/mehdiregina/Documents/jupyter/MDI343/TPTextMining', 'data', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f).read() for f in filenames_neg] #liste de textes negatifs
texts_pos = [open(f).read() for f in filenames_pos] #liste de textes positifs
texts = texts_neg + texts_pos #liste de tous les textes
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0. #output 0 pour les negatifs, 1 pour les positifs

print("%d documents" % len(texts))

Loading dataset
2000 documents


### Implémentation du classifier

#### 1) Compléter la fonction count_words qui va compter le nombre d’occurrences de chaque mot dans une liste de string et renvoyer le vocabulaire

In [7]:
def count_words(texts):
    """Vectorize text : return count of each word in the text snippets
    Parameters
    ----------
    texts : list of str
        The texts corpus

    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
        n_samples == number of documents.
        n_features == number of words in vocabulary.
    """
    words = set()
    vocabulary = dict()
    
    #1e etape get text vocabulary
    for text in texts :
        for word in re.split(r'[\n\s\.,-;\':]+',text.strip().lower()):
            words.add(word)
    
    #build a dict word-index : no index in set !
    vocabulary = dict(zip(words, range(0,len(words))))
    
    #build count matrix
    count = np.zeros((len(texts),len(words)))
    for i,text in enumerate(texts):
        for word in re.split(r'[\n\s\.,-;\':]+',text.strip().lower()):
            j = vocabulary[word]
            count[i,j]+=1
    
    return vocabulary,count 

In [10]:
voc, matrix = count_words(texts)

In [11]:
voc

{'': 0,
 'lavish': 1,
 'contraceptive': 2,
 'laughter': 3,
 'incandescent': 4,
 'near': 5,
 'helgenberger': 6,
 'juxtaposition': 7,
 'puppeteering': 8,
 'dermot': 9,
 'communicative': 10,
 'fuentes': 11,
 'sickened': 12,
 'kubrick': 13,
 'engenders': 14,
 'protestations': 15,
 'heisted': 16,
 'whose': 17,
 'tricks': 18,
 'lunkheads': 19,
 'estella': 20,
 'wage': 21,
 'innocent': 22,
 'grueling': 23,
 'capital': 24,
 'transfer': 25,
 'flourishes': 26,
 'hugged': 27,
 'display': 28,
 'uneventful': 29,
 'lauper': 30,
 'spottiswoode': 31,
 'looming': 32,
 'ginty': 33,
 'intent': 34,
 'balloon': 35,
 'irresistable': 36,
 'prospected': 37,
 'fine': 38,
 'pheiffer': 39,
 'integration': 40,
 'deck': 41,
 'mourn': 42,
 'mushroom': 43,
 'superb': 44,
 'unremittingly': 45,
 'tonino': 46,
 'skins': 47,
 'carriage': 48,
 'craftsmen': 49,
 'mu': 50,
 'searles': 51,
 'hawkins': 52,
 'bite': 53,
 'valiantly': 54,
 'terl': 55,
 'easter': 56,
 'prosperous': 57,
 'kureishi': 58,
 'naive': 59,
 'deconstru

In [12]:
matrix.shape

(2000, 39682)

#### 2) Expliquer comment les classes positives et négatives ont été assignées sur les critiques de films

This section describes how we determined whether a review was positive
or negative.

The original html files do not have consistent formats -- a review may
not have the author's rating with it, and when it does, the rating can
appear at different places in the file in different forms.  We only
recognize some of the more explicit ratings, which are extracted via a
set of ad-hoc rules.  In essence, a file's classification is determined
based on the first rating we were able to identify.


- In order to obtain more accurate rating decisions, the maximum
	rating must be specified explicitly, both for numerical ratings
	and star ratings.  ("8/10", "four out of five", and "OUT OF
	****: ***" are examples of rating indications we recognize.)

- With a five-star system (or compatible number systems):
	three-and-a-half stars and up are considered positive, 
	two stars and below are considered negative.
- With a four-star system (or compatible number system):
	three stars and up are considered positive, 
	one-and-a-half stars and below are considered negative.  
- With a letter grade system:
	B or above is considered positive,
	C- or below is considered negative.


#### 3) Compléter la classe NB pour qu’elle implémente le classifieur Naive Bayes en vous appuyant sur le pseudo-code

J'ai fait le choix d'inclure la création de matrix count per document dans mon classifier. Il prendra en input le corpus de textes et appellera la méthode count()

In [35]:
class NB(BaseEstimator, ClassifierMixin):
    def __init__(self):
        self.prior = None
        self.condprob = None
        self.vocab = None
        self.matrix = None
        pass

    def fit(self, X, y):
        #effectuer le wordcount
        self.vocab, self.matrix = count_words(X)
        
        y_0 = y[y==0]
        y_1 = y[y==1]
        matrix_0 = self.matrix[y==0]
        matrix_1 = self.matrix[y==1]
        
        #compute prior probability : P(y=k)
        prior_y_0 = len(y_0)/len(y)
        prior_y_1 = 1 -  prior_y_0
        
        #Class y=0
        cond_prob_0 = np.zeros(self.matrix.shape[1]) #1 vector of probabilities for all the words
        #somme du nombre d'occurences du mot +1
        sum_on_word_0 = matrix_0.sum(axis=0) + 1
        #somme de la somme du nombre d'occurences sur chaque mot +1
        total_sum_on_0 = (matrix_0.sum(axis=0) + 1).sum()
        #P(mot|Y=0)
        cond_prob_0 = sum_on_word_0 / total_sum_on_0
        
        #Class y=1
        cond_prob_1 = np.zeros(self.matrix.shape[1]) #1 vector of probabilities for all the words
        #somme du nombre d'occurences du mot +1
        sum_on_word_1 = matrix_1.sum(axis=0) + 1
        #somme de la somme du nombre d'occurences sur chaque mot +1
        total_sum_on_1 = (matrix_1.sum(axis=0) + 1).sum()
        #P(mot|Y=1)
        cond_prob_1 = sum_on_word_1 / total_sum_on_1
        
        self.prior = np.array((prior_y_0,prior_y_1))
        self.condprob = np.vstack((cond_prob_0,cond_prob_1))
        return self

    
    def predict(self, X):
        vocab_test,matrix_test = count_words(X)
        #gardons uniquement les mots en commun entre le test et le train
        #recuperons les mots de chacun dans un set et gardons l'intersection des mots
        keys_train = self.vocab.keys()
        keys_test = vocab_test.keys()
        keys_intersect = keys_train & keys_test
        
        #recuperons les indices associées pour le train & test dans des listes
        idx_keys_train = []
        idx_keys_test = []
        for key in keys_intersect:
            idx_keys_train.append(self.vocab[key])
            idx_keys_test.append(vocab_test[key])
        
        #renvoyer les cond_probabilities et le count du test associé aux indices
        condprob_new = self.condprob[:,idx_keys_train]
        matrix_test = matrix_test[:,idx_keys_test]
        
        #classe y = 0
        #J'ajoute la prior proba
        score_0 = np.zeros(matrix_test.shape[0])
        score_0+=np.log(self.prior[0])
        
        #add cond_prob*occurence
        prod = np.dot(matrix_test,np.log(condprob_new[0]))
        score_0+=prod
        
        
        #classe y = 1
        #J'ajoute la prior proba
        score_1 = np.zeros(matrix_test.shape[0])
        score_1+=np.log(self.prior[1])
        
        #cond_prob matrix
        #prod = np.dot(matrix_test,np.log(condprob_new[1]))
        prod = np.dot(matrix_test,np.log(condprob_new[1]))
        score_1+=prod
        
        y_predict = np.zeros(matrix_test.shape[0])
        y_predict[score_1>score_0] = 1
        return y_predict

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

In [36]:
nb = NB()
nb.fit(texts,y)
#train error 
#nb.score(texts,y)

NB()

#### 4) Evaluer les performances de votre classifieur en cross-validation 5-folds.

In [37]:
#METHOD 2 WITH cross_val_score()
res = cross_val_score(nb,texts,y,cv=5)
print(res, "moyenne : ", res.mean())

[ 0.805   0.8225  0.8075  0.8275  0.7875] moyenne :  0.81


#### 5) Modifiez la fonction count_words pour qu’elle ignore les “stop words” dans le fichier data/english.stop. Les performances sont-elles améliorées

In [38]:
# Read stop word file in a list
with open('/Users/mehdiregina/Documents/jupyter/MDI343/TPTextMining/data/english.stop','r') as text_file:
    stop_words = text_file.read().split()

In [39]:
#check de la liste des stops words donnée
stop_words

['a',
 "a's",
 'able',
 'about',
 'above',
 'according',
 'accordingly',
 'across',
 'actually',
 'after',
 'afterwards',
 'again',
 'against',
 "ain't",
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apart',
 'appear',
 'appreciate',
 'appropriate',
 'are',
 "aren't",
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'available',
 'away',
 'awfully',
 'b',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'believe',
 'below',
 'beside',
 'besides',
 'best',
 'better',
 'between',
 'beyond',
 'both',
 'brief',
 'but',
 'by',
 'c',
 "c'mon",
 "c's",
 'came',
 'can',
 "can't",
 'cannot',
 'cant',
 'cause',
 'causes',
 'certain',
 'certainly',
 'changes',
 'clearly',
 'co',
 'com',
 'come',
 'c

In [40]:
def gen_new_text (texts, stop_words):
    """Génère un nouveau corpus de texte sans les stops words"""
    set_stop_words = set(stop_words)
    new_texts = []
    for text in texts:
        word_text = []
        for word in text.split():
            if word not in stop_words:
                word_text.append(word)
        new_texts.append(" ".join(word_text))
    return new_texts

In [41]:
new_texts = gen_new_text(texts,stop_words)

In [42]:
nb.fit(new_texts,y)
res = cross_val_score(nb,new_texts,y,cv=5)
print(res, "moyenne : ", res.mean())

[ 0.795  0.815  0.8    0.83   0.765] moyenne :  0.801


On observe pas d'amélioration après exclusion de la liste des stop words. Il se peut que certains mots de cette liste étaient des features utiles à l'explication de la classification des textes comme 'awfully','appreciate', tandis que d'autres étaient effectivement négligeables. Au global cette liste de stop_word n'améliore pas l'accuracy du modèle, il serait néanmoins intéressant de la retravailler pour l'adapter à notre cas.

### Utilisation de sklearn

In [93]:
#import
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from nltk import SnowballStemmer, pos_tag, word_tokenize

#### 1) Comparer votre implémentation avec scikitlearn

In [99]:
#Utilisation d'une pipeline
pipeline_nb = Pipeline([('count_vec',CountVectorizer()),('nb_skl',MultinomialNB())])
#essai uniquement avec unigram et bigram à cause du cout computationnel
parameters = {'count_vec__analyzer':['word','char', 'char_wb'],'count_vec__stop_words':[None,'english'],
             'count_vec__ngram_range': [(1, 1),(1, 2)]} 
grid_cv = GridSearchCV(pipeline_nb,param_grid=parameters,cv=5)

In [100]:
grid_cv.fit(texts,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('nb_skl', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'count_vec__analyzer': ['word', 'char', 'char_wb'], 'count_vec__stop_words': [None, 'english'], 'count_vec__ngram_range': [(1, 1), (1, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [101]:
grid_cv.best_score_

0.83050000000000002

In [102]:
grid_cv.best_params_

{'count_vec__analyzer': 'word',
 'count_vec__ngram_range': (1, 2),
 'count_vec__stop_words': None}

A l'issue du grid search on constate qu'on obtient une meilleure accuracy que l'estimateur maison en travaillant avec des unigrams et bi-grams (1,2) de mots (analyzer = word) et en ne prenant pas en compte de liste de stop words. On pourrait imaginer obtenir encore de meilleurs scores en testant des n-grams plus grand, je ne l'ai néanmoins pas fait pour des raisons computationnelles.
L'avantage du n-grams (à partir de 2) est qu'il permet de capturer un sens contrairement à l'unigram. On va en effet tenir compte des n-1 mots précédent le mot observé. On pourrait donc penser intuitivement que plus le n-gram est grand mieux on va capturer le sens jusqu'à une certaine limite ou on ne pourra plus généraliser notre modèle car il sera devenu trop spécifique à notre train set.


A noter que si on considère les mêmes paramètres que l'estimateur maison, c'est à dire avec des unigrams de mots, on obtient une accuracy similaire néanmoins l'exécution est plus rapide.

#### 2) Tester un autre algorithme de la librairie scikitlearn

In [103]:
liste_pipeline = []
liste_pipeline.append(Pipeline([('count_vec',CountVectorizer()),('linear_svc',LinearSVC())]))
liste_pipeline.append(Pipeline([('count_vec',CountVectorizer()),('log_reg',LogisticRegression())]))

liste_score = []
liste_param = []
res = []

for pip in liste_pipeline:
    grid = GridSearchCV(pip,param_grid=parameters,cv=5)
    grid.fit(texts,y)
    liste_score.append(grid.best_score_)
    liste_param.append(grid.best_params_)
    res = list(zip(liste_param,liste_score))

res

[({'count_vec__analyzer': 'word',
   'count_vec__ngram_range': (1, 2),
   'count_vec__stop_words': None},
  0.84999999999999998),
 ({'count_vec__analyzer': 'word',
   'count_vec__ngram_range': (1, 2),
   'count_vec__stop_words': None},
  0.85250000000000004)]

On constate que le Naive Bayes a une performance (accuracy) inférieure aux Linar SVC(0.850) et à la regression logistique(0.853). Cela peut être causé par l'hypothèse forte du model génératif Naive Bayes à savoir que P(X|y=k) est égale aux produits des probabilités marginales de chaque feautre, les features sont donc considérées indépendantes. Nous savons que l'apparition d'un mot est dépendante de celle des mots qui le précède, on peut donc se demander si les features d'occurences des mots sont toutes indépendantes entre elles.

Les classes sont parfaitement équilibrées par conséquent l'utilisation de la regression logistique est appropriée. De plus la méthode de sklearn comporte par défaut une pénalisation l2 qui permet à l'algo de rester stable malgré la grande dimension. Il serait intéressant d'observer les résultats avec une pénalisation parcimonieuse comme l1. Le bon score de la logistic regression montre que l'espace des critiques positives et négatives est séparable par un hyperplan.

En outre le SVM reste stable même si le dataset est en grande dimension. Sachant qu'un classifier linéaire (log.reg) a une bonne accuracy, on peut se limiter au linearSVC qui sera moins gourmand en temps de calculs.

#### 3) Utiliser la librairie NLTK afin de procéder à une racinisation (stemming).

In [104]:
print(" ".join(SnowballStemmer.languages))

arabic danish dutch english finnish french german hungarian italian norwegian porter portuguese romanian russian spanish swedish


In [82]:
#Racinisation de notre corpus de text
stemmer = SnowballStemmer("english")
texts_stemmed = []
for text in texts:
    words_list = re.split(r'[\n\s\.,-;\':]+',text.strip())
    words_list_stem = [stemmer.stem(word) for word in words_list]
    texts_stemmed.append(" ".join(words_list_stem))

In [83]:
#verification
texts_stemmed[0]

'plot two teen coupl go to a church parti drink and then drive they get into an accid one of the guy die but his girlfriend continu to see him in her life and has nightmar what s the deal ? watch the movi and " sorta " find out critiqu a mind fuck movi for the teen generat that touch on a veri cool idea but present it in a veri bad packag which is what make this review an even harder one to write sinc i general applaud film which attempt to break the mold mess with your head and such ( lost highway & memento ) but there are good and bad way of make all type of film and these folk just didn t snag this one correct they seem to have taken this pretti neat concept but execut it terribl so what are the problem with the movi ? well it main problem is that it s simpli too jumbl it start off " normal " but then downshift into this " fantasi " world in which you as an audienc member have no idea what s go on there are dream there are charact come back from the dead there are other who look lik

In [111]:
def stem_word(text):
    """Retourne pour un texte donné une liste de mots splittés et stemmés"""
    words_list = re.split(r'[\n\s\.,-;\':]+',text.strip())
    return [stemmer.stem(word) for word in words_list]

In [124]:
pipeline_log = Pipeline([('count_vec',CountVectorizer(analyzer='word',tokenizer=stem_word,ngram_range=(1,2))),
                         ('log_reg',LogisticRegression())])

In [131]:
accuracy_score = cross_val_score(pipeline_log,texts,y,cv=5)
print(accuracy_score,"moyenne : ",accuracy_score.mean())

[ 0.8225  0.8325  0.82    0.8475  0.855 ] moyenne :  0.8355


Pas d'amélioration notable de l'accuracy à l'issu de la racinisation, elle permet une réduction dimensionnelle ce n'est néanmoins pas payant.

#### 4) Filtrer les mots par catégorie grammaticale (POS : Part Of Speech) et ne garder que les noms, les verbes, les adverbes et les adjectifs pour la classification.

In [97]:
import nltk
nltk.download('punkt')
texts_trie = []
for text in texts :
    text = word_tokenize(text) #etape prealable necessaire
    buffer = pos_tag(text)
    #je conserve uniquement adjectif(JJ), nom (NN), verbe (VBP), adverbes(RB)
    liste_trie = [tup[0] for tup in buffer if tup[1] in ['JJ','NN','VBP','RB']]
    texts_trie.append(" ".join(liste_trie))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [98]:
#VERIFICATION
texts_trie[0]

"plot teen go church party drink then drive get accident guys girlfriend life deal movie sorta critique mind-fuck movie teen generation very cool idea very bad package review even i generally applaud attempt mold mess head such highway memento are good bad just n't correctly seem pretty neat concept terribly are movie well main problem simply too normal then fantasy world audience member have idea are are back dead are look dead are strange are are looooot chase are weird happen simply not now personally do n't film now then same clue over again i get kind while film problem obviously big secret completely final do even meantime not really sad part arrow dig actually half-way point strangeness little bit sense still n't film entertaining i bottom line always sure audience even are secret password world i mean melissa sagemiller away movie just plain lazy okay get are do n't are do really over again different further insight strangeness movie apparently studio film away director 've pre

In [127]:
def tag_tokenizer(text):
    """Retourne pour un texte donné une liste de mots splittés et triés"""
    text = word_tokenize(text)
    buffer = pos_tag(text)
    return [tup[0] for tup in buffer if tup[1] in ['JJ','NN','VBP','RB']]

In [128]:
pipeline_log = Pipeline([('count_vec',CountVectorizer(analyzer='word',tokenizer=tag_tokenizer,ngram_range=(1,2))),
                         ('log_reg',LogisticRegression())])

accuracy_score = cross_val_score(pipeline_log,texts,y,cv=5)
print(accuracy_score,"moyenne : ",accuracy_score.mean())

[ 0.83    0.8275  0.8275  0.8425  0.8525] moyenne :  0.836


In [129]:
def stem_tag_tokenizer(text):
    """Retourne pour un texte donné une liste de mots splittés triés puis stemmé"""
    liste_trie = tag_tokenizer(text)
    return [stemmer.stem(word) for word in liste_trie]

In [130]:
pipeline_log = Pipeline([('count_vec',CountVectorizer(analyzer='word',tokenizer=stem_tag_tokenizer,ngram_range=(1,2))),
                         ('log_reg',LogisticRegression())])

accuracy_score = cross_val_score(pipeline_log,texts,y,cv=5)
print(accuracy_score,"moyenne : ",accuracy_score.mean())

[ 0.8225  0.8325  0.82    0.8475  0.855 ] moyenne :  0.8355


De même un tri suivit d'une racinisation entraine une réduction dimensionnelle non négligeable de notre dataset, néanmoins l'accuracy de notre modèle n'est pas améliorée.